## Indexing

In [1]:
# all the boilerplate code we have in docs.ipynb is now also in the docs.py file

import docs

github_data = docs.read_github_data()
parsed_data = docs.parse_data(github_data)
chunks = docs.chunk_documents(parsed_data)

In [2]:
chunks[0]

{'start': 0,
 'content': '<Note>\n  If you\'re not looking to build API reference documentation, you can delete\n  this section by removing the api-reference folder.\n</Note>\n\n## Welcome\n\nThere are two ways to build API documentation: [OpenAPI](https://mintlify.com/docs/api-playground/openapi/setup) and [MDX components](https://mintlify.com/docs/api-playground/mdx/configuration). For the starter kit, we are using the following OpenAPI specification.\n\n<Card\n  title="Plant Store Endpoints"\n  icon="leaf"\n  href="https://github.com/mintlify/starter/blob/main/api-reference/openapi.json"\n>\n  View the OpenAPI specification file\n</Card>\n\n## Authentication\n\nAll API endpoints are authenticated using Bearer tokens and picked up from the specification file.\n\n```json\n"security": [\n  {\n    "bearerAuth": []\n  }\n]\n```',
 'title': 'Introduction',
 'description': 'Example section for showcasing API endpoints',
 'filename': 'api-reference/introduction.mdx'}

In [3]:
from elasticsearch import Elasticsearch
import warnings

# Suppress version compatibility warnings
warnings.filterwarnings('ignore', category=UserWarning, module='elasticsearch')

# Create client with explicit compatibility settings
es_client = Elasticsearch(
    'http://localhost:9200',
    request_timeout=30,
    max_retries=10,
    retry_on_timeout=True
)

# Test connection and get server version
try:
    info = es_client.info()
    print(f"Connected to Elasticsearch version: {info['version']['number']}")
except Exception as e:
    print(f"Connection failed: {e}")
    print("Make sure Elasticsearch is running with: docker compose up -d") 

Connection failed: Connection error caused by: ConnectionError(Connection error caused by: NewConnectionError(<urllib3.connection.HTTPConnection object at 0x7f3647ba26f0>: Failed to establish a new connection: [Errno 111] Connection refused))
Make sure Elasticsearch is running with: docker compose up -d


In [4]:
index_settings = {
    "mappings": {
        "properties": {
            "start": {"type": "integer"},
            "content": {"type": "text"},
            "title": {"type": "text"},
            "description": {"type": "text"},
            "filename": {"type": "text"}
        }
    }
}

index_name = "evidently-docs"
es_client.indices.create(index=index_name, body=index_settings)

ConnectionError: Connection error caused by: ConnectionError(Connection error caused by: NewConnectionError(<urllib3.connection.HTTPConnection object at 0x7f3647bc83b0>: Failed to establish a new connection: [Errno 111] Connection refused))

In [ ]:
from tqdm.auto import tqdm

In [ ]:
for chunk in tqdm(chunks):
    es_client.index(index=index_name, document=chunk)

  0%|          | 0/575 [00:00<?, ?it/s]

## Retrieval

In [ ]:
from elasticsearch import Elasticsearch
import warnings

# Suppress version compatibility warnings
warnings.filterwarnings('ignore', category=UserWarning, module='elasticsearch')

# Create client with explicit compatibility settings
es_client = Elasticsearch(
    'http://localhost:9200',
    request_timeout=30,
    max_retries=10,
    retry_on_timeout=True
) 

In [ ]:
index_name = "evidently-docs"

In [ ]:
def elastic_search(query, num_results=15):
    es_query = {
        "size": num_results,
        "query": {
            "multi_match": {
                "query": query,
                "type": "best_fields",
                "fields": ["content", "filename", "title", "description"],
            }
        }
    }

    response = es_client.search(index=index_name, body=es_query)

    result_docs = []
    
    for hit in response['hits']['hits']:
        result_docs.append(hit['_source'])
    
    return result_docs

In [ ]:
search_results = elastic_search('how do I use llm-as-a-judge for evals')

In [ ]:
search_results[0]

{'start': 3000,
 'content': 's/customize_llm_judge#change-the-evaluator-llm) to see how you can select a different evaluator LLM. \n</Info>\n\n## 2.  Create the Dataset\n\nFirst, we\'ll create a toy Q&A dataset with customer support question that includes:\n\n- **Questions**. The inputs sent to the LLM app.\n- **Target responses**. The approved responses you consider accurate.\n- **New responses**. Imitated new responses from the system.\n- **Manual labels with explanation**. Labels that say if response is correct or not.\n\nWhy add the labels? It\'s a good idea to be the judge yourself before you write a prompt. This helps:\n\n- Formulate better criteria. You discover nuances that help you write a better prompt.\n- Get the "ground truth". You can use it to evaluate the quality of the LLM judge.\n\nUltimately, an LLM judge is a small ML system, and it needs its own evals\\!\n\n**Generate the dataframe**. Here\'s how you can create this dataset in one go:\n\n<Accordion title="Toy data t

In [ ]:
def rag(question):
    search_results = elastic_search(question)
    ...
    